In [60]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from catboost import CatBoostClassifier, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from catboost import cv

In [2]:
#baseline 0.6502190247424962 auc

При разработке baseline решений лучше сего показал себя градиентный бустинг. Рассмотрим семейство этих алгоритмов. Кроме GradientBoostingClassifier мы можем использовать XGBoost и catboost. У нас много категориальных признаков, структуру которых мы плохо представляем. Мы не можем делать предположения о ранжировании и как-то объединять группы. Это усложняет энкодинг. catboost предназнчен для работы с категориальными признаками, и задачу энкодинга он берет на себя.

In [110]:
def get_features(df, dtype=None):
    if not dtype:
        dtype = 'category'
        
    features = df.dtypes[df.dtypes == dtype].index
    features_indx = list(range(len(features)))
    return features, features_indx

def score(estimator, pool):    
    predict = estimator.predict_proba(pool)
    score = metrics.roc_auc_score(y_true=pool.get_label(), y_score=predict[:, 1])
    print(score)

def load_data(features=None):
    train_data = pd.read_csv('train-data.csv').drop(columns='Unnamed: 0')
    train_labels = pd.read_csv('train-labels.csv').drop(columns='Unnamed: 0')
    test_data = pd.read_csv('hold-out-data.csv').drop(columns='Unnamed: 0')
    test_labels = pd.read_csv('hold-out-labels.csv').drop(columns='Unnamed: 0')
    
    cat_features_origin = train_data.iloc[: , train_data.columns.get_loc("Var191"):].columns
    num_features_origin = train_data.iloc[: , :train_data.columns.get_loc("Var191")].columns
    
    train_data[cat_features_origin] = train_data[cat_features_origin].applymap(str).astype("category")
    test_data[cat_features_origin] = test_data[cat_features_origin].applymap(str).astype("category")
    
    if features:
        features = sorted(features, key=lambda x: int(x[3:]))
        train_data = train_data[features]
        test_data = test_data[features]

    train_labels[train_labels == -1] = 0
    test_labels[test_labels == -1] = 0

    cat_features, cat_features_indx = get_features(train_data) 
    num_features, _ = get_features(train_data, dtype=np.number)   
    
    train_X = pd.concat([train_data[cat_features], train_data[num_features]], axis=1)
    test_X = pd.concat([test_data[cat_features], test_data[num_features]], axis=1)
    
    train_pool = Pool(train_X, label=train_labels, cat_features=cat_features_indx)
    test_pool = Pool(test_X, label=test_labels, cat_features=cat_features_indx)
    
    return train_pool, test_pool, train_X, test_X, train_labels, test_labels

def submit_prediction(estimator, features=None):
    X = pd.read_csv('orange_small_churn_test_data.csv')    
    cat_features = X.iloc[: , X.columns.get_loc("Var191"):].columns    
    X[cat_features] = X[cat_features].applymap(str).astype("category")
    if features:
        features = sorted(features, key=lambda x: int(x[3:]))
        X = X[features]
        
    cat_features, _ = get_features(X) 
    num_features, _ = get_features(X, dtype=np.number)   
    
    X = pd.concat([X[cat_features], X[num_features]], axis=1)
   
    pred = estimator.predict_proba(X=X)
    df = pd.DataFrame(pred[:,1], columns=['result'])
    df.index = df.index.rename('Id')
    df.to_csv('submission_test.csv', sep=',')

# catboost baseline

In [5]:
train_pool, test_pool, *_ = load_data()          
                
clf = CatBoostClassifier(
    iterations=500,
    early_stopping_rounds=20,
    eval_metric='AUC',
    random_strength=1,
    leaf_estimation_method='Newton'
)
           
clf.fit(train_pool)

0:	total: 182ms	remaining: 1m 30s
1:	total: 209ms	remaining: 52s
2:	total: 293ms	remaining: 48.6s
3:	total: 344ms	remaining: 42.6s
4:	total: 404ms	remaining: 40s
5:	total: 417ms	remaining: 34.3s
6:	total: 430ms	remaining: 30.3s
7:	total: 479ms	remaining: 29.5s
8:	total: 557ms	remaining: 30.4s
9:	total: 637ms	remaining: 31.2s
10:	total: 758ms	remaining: 33.7s
11:	total: 778ms	remaining: 31.6s
12:	total: 840ms	remaining: 31.5s
13:	total: 892ms	remaining: 31s
14:	total: 993ms	remaining: 32.1s
15:	total: 1.05s	remaining: 31.7s
16:	total: 1.15s	remaining: 32.8s
17:	total: 1.24s	remaining: 33.1s
18:	total: 1.27s	remaining: 32.1s
19:	total: 1.3s	remaining: 31.1s
20:	total: 1.33s	remaining: 30.3s
21:	total: 1.34s	remaining: 29.1s
22:	total: 1.37s	remaining: 28.5s
23:	total: 1.38s	remaining: 27.5s
24:	total: 1.42s	remaining: 27s
25:	total: 1.51s	remaining: 27.5s
26:	total: 1.62s	remaining: 28.4s
27:	total: 1.69s	remaining: 28.5s
28:	total: 1.78s	remaining: 28.9s
29:	total: 1.89s	remaining: 29.6

241:	total: 24.3s	remaining: 26s
242:	total: 24.5s	remaining: 25.9s
243:	total: 24.6s	remaining: 25.8s
244:	total: 24.7s	remaining: 25.7s
245:	total: 24.8s	remaining: 25.6s
246:	total: 24.9s	remaining: 25.5s
247:	total: 25s	remaining: 25.4s
248:	total: 25.1s	remaining: 25.3s
249:	total: 25.2s	remaining: 25.2s
250:	total: 25.3s	remaining: 25.1s
251:	total: 25.4s	remaining: 25s
252:	total: 25.4s	remaining: 24.8s
253:	total: 25.5s	remaining: 24.7s
254:	total: 25.6s	remaining: 24.6s
255:	total: 25.7s	remaining: 24.5s
256:	total: 25.8s	remaining: 24.4s
257:	total: 25.9s	remaining: 24.3s
258:	total: 26s	remaining: 24.2s
259:	total: 26.1s	remaining: 24.1s
260:	total: 26.2s	remaining: 24s
261:	total: 26.3s	remaining: 23.9s
262:	total: 26.4s	remaining: 23.8s
263:	total: 26.5s	remaining: 23.7s
264:	total: 26.6s	remaining: 23.6s
265:	total: 26.7s	remaining: 23.5s
266:	total: 26.8s	remaining: 23.4s
267:	total: 26.9s	remaining: 23.3s
268:	total: 27s	remaining: 23.2s
269:	total: 27.1s	remaining: 23.

478:	total: 51.1s	remaining: 2.24s
479:	total: 51.2s	remaining: 2.13s
480:	total: 51.3s	remaining: 2.03s
481:	total: 51.4s	remaining: 1.92s
482:	total: 51.5s	remaining: 1.81s
483:	total: 51.7s	remaining: 1.71s
484:	total: 51.8s	remaining: 1.6s
485:	total: 51.9s	remaining: 1.5s
486:	total: 52s	remaining: 1.39s
487:	total: 52.1s	remaining: 1.28s
488:	total: 52.2s	remaining: 1.18s
489:	total: 52.4s	remaining: 1.07s
490:	total: 52.5s	remaining: 962ms
491:	total: 52.6s	remaining: 855ms
492:	total: 52.7s	remaining: 748ms
493:	total: 52.8s	remaining: 641ms
494:	total: 52.9s	remaining: 534ms
495:	total: 53s	remaining: 427ms
496:	total: 53.1s	remaining: 321ms
497:	total: 53.2s	remaining: 214ms
498:	total: 53.3s	remaining: 107ms
499:	total: 53.4s	remaining: 0us


In [7]:
score(clf, test_pool)

0.6900803712672883


# greedSerchCV

1. Проанализируем feature_importance
2. Запустим поиск параметров

In [ ]:
_, test_pool, train_X, _, train_y, _ = load_data()
_, cat_features = get_cat_features(train_X)

cat_features, train_X.columns

In [13]:
_, test_pool, train_X, _, train_y, _ = load_data()
_, cat_features = get_cat_features(train_X)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)

grid = {}
grid['depth'] = [2, 6]
grid['l2_leaf_reg'] = [1, 3, 5,]     

clf = CatBoostClassifier(
    loss_function='Logloss',
    learning_rate=0.3,
    iterations=500,
    early_stopping_rounds=20,
    eval_metric='AUC',
    random_strength=1,
    leaf_estimation_method='Newton',
    boosting_type = 'Ordered',
    cat_features=cat_features,
    bootstrap_type='Bernoulli',
    one_hot_max_size=2,
)

grid = GridSearchCV(estimator=clf, param_grid=grid, cv=cv, scoring='roc_auc', n_jobs=-1, error_score='raise')
grid.fit(train_X, train_y)

print(grid.best_estimator_)
print(grid.best_params_)
print(grid.best_score_)

score(grid, test_pool)

0:	total: 40.3ms	remaining: 20.1s
1:	total: 73ms	remaining: 18.2s
2:	total: 89ms	remaining: 14.7s
3:	total: 130ms	remaining: 16.1s
4:	total: 177ms	remaining: 17.5s
5:	total: 224ms	remaining: 18.4s
6:	total: 284ms	remaining: 20s
7:	total: 318ms	remaining: 19.6s
8:	total: 356ms	remaining: 19.4s
9:	total: 402ms	remaining: 19.7s
10:	total: 446ms	remaining: 19.8s
11:	total: 492ms	remaining: 20s
12:	total: 542ms	remaining: 20.3s
13:	total: 581ms	remaining: 20.2s
14:	total: 626ms	remaining: 20.2s
15:	total: 672ms	remaining: 20.3s
16:	total: 719ms	remaining: 20.4s
17:	total: 767ms	remaining: 20.5s
18:	total: 812ms	remaining: 20.5s
19:	total: 857ms	remaining: 20.6s
20:	total: 903ms	remaining: 20.6s
21:	total: 942ms	remaining: 20.5s
22:	total: 981ms	remaining: 20.3s
23:	total: 1.03s	remaining: 20.4s
24:	total: 1.07s	remaining: 20.4s
25:	total: 1.12s	remaining: 20.4s
26:	total: 1.16s	remaining: 20.3s
27:	total: 1.19s	remaining: 20.1s
28:	total: 1.23s	remaining: 20s
29:	total: 1.28s	remaining: 20s

243:	total: 11.8s	remaining: 12.4s
244:	total: 11.9s	remaining: 12.3s
245:	total: 11.9s	remaining: 12.3s
246:	total: 12s	remaining: 12.2s
247:	total: 12s	remaining: 12.2s
248:	total: 12.1s	remaining: 12.2s
249:	total: 12.1s	remaining: 12.1s
250:	total: 12.2s	remaining: 12.1s
251:	total: 12.2s	remaining: 12s
252:	total: 12.3s	remaining: 12s
253:	total: 12.3s	remaining: 11.9s
254:	total: 12.3s	remaining: 11.9s
255:	total: 12.4s	remaining: 11.8s
256:	total: 12.5s	remaining: 11.8s
257:	total: 12.5s	remaining: 11.7s
258:	total: 12.5s	remaining: 11.7s
259:	total: 12.6s	remaining: 11.6s
260:	total: 12.6s	remaining: 11.6s
261:	total: 12.7s	remaining: 11.5s
262:	total: 12.7s	remaining: 11.5s
263:	total: 12.8s	remaining: 11.4s
264:	total: 12.8s	remaining: 11.4s
265:	total: 12.9s	remaining: 11.3s
266:	total: 12.9s	remaining: 11.3s
267:	total: 13s	remaining: 11.2s
268:	total: 13s	remaining: 11.2s
269:	total: 13.1s	remaining: 11.1s
270:	total: 13.1s	remaining: 11.1s
271:	total: 13.2s	remaining: 11s

481:	total: 23.5s	remaining: 878ms
482:	total: 23.6s	remaining: 829ms
483:	total: 23.6s	remaining: 780ms
484:	total: 23.7s	remaining: 731ms
485:	total: 23.7s	remaining: 683ms
486:	total: 23.7s	remaining: 634ms
487:	total: 23.8s	remaining: 585ms
488:	total: 23.8s	remaining: 536ms
489:	total: 23.9s	remaining: 488ms
490:	total: 23.9s	remaining: 439ms
491:	total: 24s	remaining: 390ms
492:	total: 24.1s	remaining: 342ms
493:	total: 24.1s	remaining: 293ms
494:	total: 24.2s	remaining: 244ms
495:	total: 24.2s	remaining: 195ms
496:	total: 24.2s	remaining: 146ms
497:	total: 24.3s	remaining: 97.6ms
498:	total: 24.3s	remaining: 48.8ms
499:	total: 24.4s	remaining: 0us
{'depth': 2, 'l2_leaf_reg': 5}
0.6757581864688784
0.6837853603351463


In [17]:
submit_prediction(grid)

Отчет по feature importacne показал, что не все фичи имеют одинаковый вес. Произведем отбор признаков среди всех доступных.

In [56]:
train_pool, test_pool, *_ = load_data()          
                
clf = CatBoostClassifier(
    iterations=500,
    depth=2,
    l2_leaf_reg=5,
    early_stopping_rounds=50,
    eval_metric='AUC',
    random_strength=1,
    leaf_estimation_method='Newton',
    boosting_type = 'Ordered',
    learning_rate=0.1,
)

select_features_report = clf.select_features(
    train_pool,
    eval_set=test_pool,
    features_for_select=train_pool.get_feature_names(),
    num_features_to_select=35,
    steps=5,
    algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
    shap_calc_type=EShapCalcType.Exact,
    train_final_model=True,
    logging_level='Silent',
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [111]:
train_pool, test_pool, *_ = load_data(select_features_report['selected_features_names'])

clf.fit(train_pool)
score(clf, test_pool)
submit_prediction(clf, select_features_report['selected_features_names'])

0:	total: 29.1ms	remaining: 14.5s
1:	total: 62.6ms	remaining: 15.6s
2:	total: 96ms	remaining: 15.9s
3:	total: 117ms	remaining: 14.5s
4:	total: 148ms	remaining: 14.7s
5:	total: 177ms	remaining: 14.6s
6:	total: 207ms	remaining: 14.6s
7:	total: 240ms	remaining: 14.7s
8:	total: 270ms	remaining: 14.7s
9:	total: 300ms	remaining: 14.7s
10:	total: 331ms	remaining: 14.7s
11:	total: 363ms	remaining: 14.8s
12:	total: 392ms	remaining: 14.7s
13:	total: 421ms	remaining: 14.6s
14:	total: 448ms	remaining: 14.5s
15:	total: 478ms	remaining: 14.5s
16:	total: 521ms	remaining: 14.8s
17:	total: 550ms	remaining: 14.7s
18:	total: 588ms	remaining: 14.9s
19:	total: 618ms	remaining: 14.8s
20:	total: 648ms	remaining: 14.8s
21:	total: 680ms	remaining: 14.8s
22:	total: 712ms	remaining: 14.8s
23:	total: 741ms	remaining: 14.7s
24:	total: 768ms	remaining: 14.6s
25:	total: 801ms	remaining: 14.6s
26:	total: 836ms	remaining: 14.6s
27:	total: 869ms	remaining: 14.7s
28:	total: 930ms	remaining: 15.1s
29:	total: 961ms	remain

244:	total: 8.51s	remaining: 8.86s
245:	total: 8.54s	remaining: 8.82s
246:	total: 8.58s	remaining: 8.79s
247:	total: 8.61s	remaining: 8.75s
248:	total: 8.65s	remaining: 8.71s
249:	total: 8.68s	remaining: 8.68s
250:	total: 8.71s	remaining: 8.64s
251:	total: 8.74s	remaining: 8.61s
252:	total: 8.78s	remaining: 8.57s
253:	total: 8.81s	remaining: 8.53s
254:	total: 8.84s	remaining: 8.49s
255:	total: 8.87s	remaining: 8.45s
256:	total: 8.89s	remaining: 8.41s
257:	total: 8.93s	remaining: 8.37s
258:	total: 8.96s	remaining: 8.34s
259:	total: 8.99s	remaining: 8.3s
260:	total: 9.02s	remaining: 8.26s
261:	total: 9.05s	remaining: 8.22s
262:	total: 9.09s	remaining: 8.19s
263:	total: 9.12s	remaining: 8.15s
264:	total: 9.15s	remaining: 8.11s
265:	total: 9.18s	remaining: 8.07s
266:	total: 9.21s	remaining: 8.04s
267:	total: 9.25s	remaining: 8.01s
268:	total: 9.28s	remaining: 7.97s
269:	total: 9.32s	remaining: 7.94s
270:	total: 9.35s	remaining: 7.9s
271:	total: 9.38s	remaining: 7.87s
272:	total: 9.42s	rema

485:	total: 16.5s	remaining: 474ms
486:	total: 16.5s	remaining: 440ms
487:	total: 16.5s	remaining: 406ms
488:	total: 16.6s	remaining: 372ms
489:	total: 16.6s	remaining: 339ms
490:	total: 16.6s	remaining: 305ms
491:	total: 16.6s	remaining: 271ms
492:	total: 16.7s	remaining: 237ms
493:	total: 16.7s	remaining: 203ms
494:	total: 16.7s	remaining: 169ms
495:	total: 16.8s	remaining: 135ms
496:	total: 16.8s	remaining: 101ms
497:	total: 16.8s	remaining: 67.6ms
498:	total: 16.9s	remaining: 33.8ms
499:	total: 16.9s	remaining: 0us
0.7580516934618174
